# Wykrywanie encji nazwanych z Flair

To już ostatnie laboratoria zadaniowe, w związku z tym, jeśli znajdziecie chwilę wolnego czasu, wypełnijcie proszę ankietę: https://docs.google.com/forms/d/1rHPjpL70XdXRD-ILl3AHophPNUk0AhsFus1-mtkUPsI

Pozwoli to mi poprawić laboratoria w przyszłości, z góry dziękuję :)

# Flair

Biblioteka Flair to bardzo popularne narzędzie do tagowania sekwencji. Zaintstalujmy ją

In [ ]:
!pip install flair

In [ ]:
# OPCJONALNE WSPARCIE DLA KART GRAFICZNYCH
# W colabie możemy trenować z wykorzystaniem karty graficznej, dzięki temu trening działa dużo szybciej
# Aby włączyć wsparcie dla karty graficznej musimy:
# 1. w menu 'srodowisko wykonawcze' wybrać `zmien typ srodowiska wykonawczego` i tam `akcelerator sprzętowy` = GPU
# 2. odkomentować linijki poniżej
import flair, torch
flair.device = torch.device('cpu')

# Ładowanie zbioru danych i słownika z etykietami.

**Zadanie1: (1 punkt):** Stwórz słownik etykiet z wczytanego korpusu korzystając z funkcji `make_label_dictionary()`. W naszym zbiorze, etykiety do wykrycia występują w kolumnie `ner`, której identyfikator został zapisany w linijce 6. Tutorial: https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md może okazać się pomocny.

Efektem działania powinna być lista etykiet np:
`Dictionary with 20 tags: <unk>, Variable, Class, Application, User_Interface_Element, Code_Block, Language, Function, Data_Structure, Library, Data_Type, File_Type, File_Name, Version, HTML_XML_Tag, Device, Operating_System, Website, User_Name, Algorithm`

In [25]:
# zbiór otagowanych postów na Stacku
from flair.datasets import NER_ENGLISH_STACKOVERFLOW

# pobieramy korpus i zmniejszamy jego wielkość
corpus = NER_ENGLISH_STACKOVERFLOW().downsample(0.1)
# usuwamy puste zdania
corpus.filter_empty_sentences()

# identyfikator pod którym możemy dostać typy etykiet
label_type = 'ner'

2022-05-26 19:54:27,485 File train has 741 questions and 897 answers.
2022-05-26 19:54:27,497 File test has 249 questions and 315 answers.
2022-05-26 19:54:27,507 File dev has 247 questions and 289 answers.
2022-05-26 19:54:27,508 Reading data from C:\Users\Hououin Kyouma\.flair\datasets\ner_english_stackoverflow
2022-05-26 19:54:27,508 Train: C:\Users\Hououin Kyouma\.flair\datasets\ner_english_stackoverflow\train.txt
2022-05-26 19:54:27,508 Dev: C:\Users\Hououin Kyouma\.flair\datasets\ner_english_stackoverflow\dev.txt
2022-05-26 19:54:27,509 Test: C:\Users\Hououin Kyouma\.flair\datasets\ner_english_stackoverflow\test.txt
2022-05-26 19:54:31,497 Filtering empty sentences
2022-05-26 19:54:31,512 Corpus: 926 train + 294 dev + 311 test sentences
2022-05-26 19:54:31,514 Computing label dictionary. Progress:


926it [00:00, 70566.79it/s]

2022-05-26 19:54:31,529 Dictionary created for label 'ner' with 20 values: Application (seen 138 times), Class (seen 125 times), User_Interface_Element (seen 117 times), Variable (seen 94 times), Language (seen 84 times), Library (seen 81 times), Code_Block (seen 79 times), Data_Structure (seen 71 times), Function (seen 67 times), Data_Type (seen 38 times), Version (seen 29 times), File_Type (seen 27 times), HTML_XML_Tag (seen 24 times), Device (seen 23 times), File_Name (seen 22 times), Operating_System (seen 12 times), Website (seen 11 times), User_Name (seen 6 times), Algorithm (seen 5 times)
Etykiety do wykrycia Dictionary with 20 tags: <unk>, Application, Class, User_Interface_Element, Variable, Language, Library, Code_Block, Data_Structure, Function, Data_Type, Version, File_Type, HTML_XML_Tag, Device, File_Name, Operating_System, Website, User_Name, Algorithm


In [26]:
# TODO
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(f'Etykiety do wykrycia {label_dict}')

2022-05-26 19:54:47,224 Computing label dictionary. Progress:


926it [00:00, 68275.60it/s]

2022-05-26 19:54:47,240 Dictionary created for label 'ner' with 20 values: Application (seen 138 times), Class (seen 125 times), User_Interface_Element (seen 117 times), Variable (seen 94 times), Language (seen 84 times), Library (seen 81 times), Code_Block (seen 79 times), Data_Structure (seen 71 times), Function (seen 67 times), Data_Type (seen 38 times), Version (seen 29 times), File_Type (seen 27 times), HTML_XML_Tag (seen 24 times), Device (seen 23 times), File_Name (seen 22 times), Operating_System (seen 12 times), Website (seen 11 times), User_Name (seen 6 times), Algorithm (seen 5 times)
Etykiety do wykrycia Dictionary with 20 tags: <unk>, Application, Class, User_Interface_Element, Variable, Language, Library, Code_Block, Data_Structure, Function, Data_Type, Version, File_Type, HTML_XML_Tag, Device, File_Name, Operating_System, Website, User_Name, Algorithm


# Embeddingi

W narzędziu Flair możemy bardzo prosto składać ze sobą różne embeddingi.

**Zad2 (2 punkty):** Zapoznaj się z działaniem `StackedEmbeddings` opisanego w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md i zbuduj embeddingi zawierające reprezentacje pochodzące zarówno z Glove jak i Flairowe, oparte na `news-forward`.

In [35]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.data import Sentence

# TODO
embeddings = StackedEmbeddings([
  FlairEmbeddings('news-forward'),
  WordEmbeddings('glove')
])

In [38]:
sentence = Sentence('fn foo() -> void {}')

for token in sentence:
  print(token.value, token.score)

AttributeError: 'Token' object has no attribute 'value'

# Tagger i trainer

**Zadanie 3 (2 punkty)** Bazując na treściach opisanych w https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md przygotuj obiekt `SequenceTagger`, którego rozmiar wartswy ukrytej wyniesie 256. Do obiektu tego przekażemy stworzone wcześniej embeddingi, słownik `label_dict` i nazwę kolumny z etykietą ze zmiennej `label_type`. Ustawmy `use_crf` na True.

Przygotuj obiekt `ModelTrainer`, który przyjmie zarówno nasz korpus jak i stworzony przed chwilą `SequenceTagger`.

In [37]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


# TODO
tagger = SequenceTagger(hidden_size=256, embeddings=embeddings, tag_dictionary=label_dict, tag_type=label_type)

# TODO
trainer = ModelTrainer(tagger, corpus)

#stworzony trainer możemy zacząć trenować!
trainer.train('resources/taggers/example-upos', learning_rate=0.1, mini_batch_size=32, max_epochs=5)

# a kiedy wytrenujemy, wczytujemy najlepszy model.
model = SequenceTagger.load('resources/taggers/example-upos/final-model.pt')

2022-05-26 20:03:31,262 SequenceTagger predicts: Dictionary with 77 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Variable, B-Variable, E-Variable, I-Variable, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Function, B-Function, E-Function, I-Function, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-Version, B-Version, E-Version, I-Version, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-HTML_XML_Tag
2022-05-26 20:03:31,304 ----------------------------------------------------------------------------------------------------
2022-05-26 20:03:31,305 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddi

100%|██████████| 10/10 [00:24<00:00,  2.43s/it]

2022-05-26 20:05:54,759 Evaluating as a multi-label problem: False
2022-05-26 20:05:54,767 DEV : loss 0.5774096846580505 - f1-score (micro avg)  0.0
2022-05-26 20:05:54,785 BAD EPOCHS (no improvement): 0
2022-05-26 20:05:54,786 ----------------------------------------------------------------------------------------------------


2022-05-26 20:05:57,872 epoch 2 - iter 2/29 - loss 0.63576726 - samples/sec: 20.76 - lr: 0.100000
2022-05-26 20:05:59,557 epoch 2 - iter 4/29 - loss 0.66468348 - samples/sec: 37.99 - lr: 0.100000
2022-05-26 20:06:02,122 epoch 2 - iter 6/29 - loss 0.68594443 - samples/sec: 24.97 - lr: 0.100000
2022-05-26 20:06:04,469 epoch 2 - iter 8/29 - loss 0.70621169 - samples/sec: 27.31 - lr: 0.100000
2022-05-26 20:06:07,058 epoch 2 - iter 10/29 - loss 0.68488611 - samples/sec: 24.73 - lr: 0.100000
2022-05-26 20:06:09,470 epoch 2 - iter 12/29 - loss 0.67300155 - samples/sec: 26.54 - lr: 0.100000
2022-05-26 20:06:15,623 epoch 2 - iter 14/29 - loss 0.66140667 - samples/sec: 10.40 - lr: 0.100000
2022-05-26 20:06:18,114 epoch 2 - iter 16/29 - loss 0.67685919 - samples/sec: 25.71 - lr: 0.100000
2022-05-26 20:06:19,980 epoch 2 - iter 18/29 - loss 0.66057448 - samples/sec: 34.32 - lr: 0.100000
2022-05-26 20:06:21,521 epoch 2 - iter 20/29 - loss 0.65636615 - samples/sec: 41.55 - lr: 0.100000
2022-05-26 20:

100%|██████████| 10/10 [00:01<00:00,  5.32it/s]

...

2022-05-26 20:08:33,023 Evaluating as a multi-label problem: False
2022-05-26 20:08:33,031 DEV : loss 0.43200454115867615 - f1-score (micro avg)  0.1542
2022-05-26 20:08:33,044 BAD EPOCHS (no improvement): 0
2022-05-26 20:08:33,047 saving best model


2022-05-26 20:08:34,268 ----------------------------------------------------------------------------------------------------
2022-05-26 20:08:34,269 loading file resources\taggers\example-upos\best-model.pt
2022-05-26 20:08:34,725 SequenceTagger predicts: Dictionary with 79 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Variable, B-Variable, E-Variable, I-Variable, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Function, B-Function, E-Function, I-Function, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-Version, B-Version, E-Version, I-Version, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-HTML_XML_Tag


100%|██████████| 10/10 [00:31<00:00,  3.19s/it]

2022-05-26 20:09:06,766 Evaluating as a multi-label problem: False
2022-05-26 20:09:06,774 0.1646	0.069	0.0972	0.0602
2022-05-26 20:09:06,774 
Results:
- F-score (micro) 0.0972
- F-score (macro) 0.0348
- Accuracy 0.0602

By class:
                        precision    recall  f1-score   support

           Application     0.1148    0.1273    0.1207        55
                 Class     0.2973    0.2245    0.2558        49
              Language     0.1944    0.3182    0.2414        22
               Library     0.0714    0.0312    0.0435        32
              Variable     0.0000    0.0000    0.0000        40
            Code_Block     0.0000    0.0000    0.0000        37
User_Interface_Element     0.0000    0.0000    0.0000        32
              Function     0.0000    0.0000    0.0000        23
             File_Name     0.0000    0.0000    0.0000        22
               Version     0.0000    0.0000    0.0000        16
             File_Type     0.0000    0.0000    0.0000        11


2022-05-26 20:09:07,260 SequenceTagger predicts: Dictionary with 79 tags: O, S-Application, B-Application, E-Application, I-Application, S-Class, B-Class, E-Class, I-Class, S-User_Interface_Element, B-User_Interface_Element, E-User_Interface_Element, I-User_Interface_Element, S-Variable, B-Variable, E-Variable, I-Variable, S-Language, B-Language, E-Language, I-Language, S-Library, B-Library, E-Library, I-Library, S-Code_Block, B-Code_Block, E-Code_Block, I-Code_Block, S-Data_Structure, B-Data_Structure, E-Data_Structure, I-Data_Structure, S-Function, B-Function, E-Function, I-Function, S-Data_Type, B-Data_Type, E-Data_Type, I-Data_Type, S-Version, B-Version, E-Version, I-Version, S-File_Type, B-File_Type, E-File_Type, I-File_Type, S-HTML_XML_Tag


# Predykcja z udziałem modelu

Jeśli model został wytrenowany, poniżej znajdziemy fragment kodu, który może wykryć encje w zdaniach.

In [40]:
from flair.data import Sentence

# Jeśli nasz model się wyuczył, powinien wykryć Python jako język.
# Uwaga, ponieważ pracujemy na niewielkim podzbiorze zbioru danych (downsample(0.1) próbkuje 10%),
# otrzymywane wyniki mogą być kiepskiej jakości, najlepiej zwiększyść ilość danych
# jeśli pracujemy w domu.
# stwórz obiekt zdania
sentence = Sentence('fn foo() -> void {}')
# wykryj encje nazwane
model.predict(sentence)
# wyświetl zdanie i wykryte w nim encje
print(sentence.to_tagged_string())

Sentence: "fn foo () -> void {}" → ["fn"/Library, "foo"/File_Type, "()"/Application]
